In [ ]:
import numpy as np
import time

print("------------------------")
print("Sudoku Solver")
print("------------------------")
grid = np.zeros((9,9), dtype='int')

while True:
    try:
        print("Please Enter Sudoku Grid:")
        for i in range(9):
            stringrow = input("")
            for j,character in enumerate(stringrow):
                grid[i][j] = int(character)
    except:
        print("Data Format Incorrect..")
        continue
    break
    
#Getting the set of numbers within a number's group------------------------------------
dx = np.array([0,1,2,0,1,2,0,1,2])
dy = np.array([0,0,0,1,1,1,2,2,2])
def getgroupset(grid,groupx,groupy):
    s = set()
    tlx = groupx * 3                                                    #The x coordinate of the top left square in group
    tly = groupy * 3                                                    #The y coordinate of the top left square in group
    for i in range(9):
        s.add(grid[tly+dy[i]][tlx+dx[i]])
    return s

#Function to check if a sudoku grid is solved------------------------------------------
def check(grid):
    s = set()
    #Check rows
    for row in grid:
        s.clear()
        for value in row:
            s.add(value)
        if list(s) != [1,2,3,4,5,6,7,8,9]:
            return False
    #Check columns
    grid = grid.transpose()
    for column in grid:
        s.clear()
        for value in column:
            s.add(value)
        if list(s) != [1,2,3,4,5,6,7,8,9]:
            return False
    grid = grid.transpose()

    #Check groups (3x3 squares)
    for groupx in range(3):
        for groupy in range(3):
            s = getgroupset(grid, groupx, groupy)
            if list(s) != [1,2,3,4,5,6,7,8,9]:
                return False
    return True

#Recursive function to solve the grid--------------------------------------------------
counter =  -1
def solve(grid):
    if check(grid):                                                     #Return if solved (base case)
        return True, grid

    global counter
    counter+= 1
    min_no = 10                                                         #The min number of possibilities
    fullset = set([1,2,3,4,5,6,7,8,9])                                  #Set of numbers in each row, column and group
    tgrid = grid.transpose()                                            #Transposed Grid
    for y, row in enumerate(grid):
        for x, value in enumerate(row):
            if value == 0:
                setx = set(row)
                sety = set(tgrid[x])
                setgroup = getgroupset(grid, int(x/3), int(y/3))
                possible = fullset - setx -sety - setgroup              #Set of possible values
                
                if len(possible) < min_no:
                    miny = y
                    minx = x
                    min_pos = possible
                    min_no = len(possible)
                    
    #print(min_pos)
    #print(minx, miny)                                             
    for possibility in min_pos:
        grid[miny][minx] = possibility
        solved, solved_grid = solve(np.copy(grid))                      #Important!! Ensure a copy of the array is passed into the recursive function
        if solved:
            return True, solved_grid
    return False, False
    
start = time.time()
solved, grid = solve(grid)
if solved:
    print("Solved!")
    print("Time Taken:", str(time.time()-start) + 's')
    print("No. of substitutions:", counter)
    print (grid)
else:
    print("Seems actually impossible...")

------------------------
Sudoku Solver
------------------------
Please Enter Sudoku Grid:
240000009
090031000
001590080
000000076
000803000
960000000
030045100
000620090
600000052
Solved!
Time Taken: 0.044980764389038086s
No. of substitutions: 64
[[2 4 5 7 8 6 3 1 9]
 [8 9 6 4 3 1 5 2 7]
 [3 7 1 5 9 2 6 8 4]
 [4 1 3 2 5 9 8 7 6]
 [5 2 7 8 6 3 9 4 1]
 [9 6 8 1 7 4 2 3 5]
 [7 3 2 9 4 5 1 6 8]
 [1 5 4 6 2 8 7 9 3]
 [6 8 9 3 1 7 4 5 2]]


In [ ]:
Possible
Inputs

240000009
090031000
001590080
000000076
000803000
960000000
030045100
000620090
600000052

764000100
000070806
809604030
000800063
090351020
580006000
050902301
902010000
006000289

063400000
200000090
000870106
000040300
420000057
007080000
902056000
050000009
000008570


In [ ]:
import numpy as np

def generate_sudoku(mask_rate=0.5):
    while True:
        n = 9
        m = np.zeros((n, n), np.int)
        rg = np.arange(1, n + 1)
        m[0, :] = np.random.choice(rg, n, replace=False)
        try:
            for r in range(1, n):
                for c in range(n):
                    col_rest = np.setdiff1d(rg, m[:r, c])
                    row_rest = np.setdiff1d(rg, m[r, :c])
                    avb1 = np.intersect1d(col_rest, row_rest)
                    sub_r, sub_c = r//3, c//3
                    avb2 = np.setdiff1d(np.arange(0, n+1), m[sub_r*3:(sub_r+1)*3, sub_c*3:(sub_c+1)*3].ravel())
                    avb = np.intersect1d(avb1, avb2)
                    m[r, c] = np.random.choice(avb, size=1)
            break
        except ValueError:
            pass
    print("Answer:\n", m)
    mm = m.copy()
    mm[np.random.choice([True, False], size=m.shape, p=[mask_rate, 1 - mask_rate])] = 0
    print("\nMasked anwser:\n", mm)
    np.savetxt("./puzzle.csv", mm, "%d", delimiter=",")
    return mm


def solve(m):
    if isinstance(m, list):
        m = np.array(m)
    elif isinstance(m, str):
        m = np.loadtxt(m, dtype=np.int, delimiter=",")
    rg = np.arange(m.shape[0]+1)
    while True:
        mt = m.copy()
        while True:
            d = []
            d_len = []
            for i in range(m.shape[0]):
                for j in range(m.shape[1]):
                    if mt[i, j] == 0:
                        possibles = np.setdiff1d(rg, np.union1d(np.union1d(mt[i, :], mt[:, j]), mt[3*(i//3):3*(i//3+1), 3*(j//3):3*(j//3+1)]))
                        d.append([i, j, possibles])
                        d_len.append(len(possibles))
            if len(d) == 0:
                break
            idx = np.argmin(d_len)
            i, j, p = d[idx]
            if len(p) > 0:
                num = np.random.choice(p)
            else:
                break
            mt[i, j] = num
            if len(d) == 0:
                break
        if np.all(mt != 0):
            break

    print("\nTrail:\n", mt)
    return mt


def check_solution(m):
    if isinstance(m, list):
        m = np.array(m)
    elif isinstance(m, str):
        m = np.loadtxt(m, dtype=np.int, delimiter=",")
    set_rg = set(np.arange(1, m.shape[0] + 1))
    no_good = False
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            r1 = set(m[3 * (i // 3):3 * (i // 3 + 1), 3 * (j // 3):3 * (j // 3 + 1)].ravel()) == set_rg
            r2 = set(m[i, :]) == set_rg
            r3 = set(m[:, j]) == set_rg
            if not (r1 and r2 and r3):
                no_good = True
                break
        if no_good:
            break
    if no_good:
        print("\nChecked: not good")
    else:
        print("\nChecked: OK")


if __name__ == "__main__":
    puzzle = generate_sudoku(mask_rate=0.7)
    solved = solve(puzzle)
    check_solution(solved)

    print("\nSolve in code:")
    solve([
        [0, 5, 0, 0, 6, 7, 9, 0, 0],
        [0, 2, 0, 0, 0, 8, 4, 0, 0],
        [0, 3, 0, 9, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 2, 1, 0, 4],
        [0, 0, 0, 6, 0, 9, 0, 0, 0],
        [0, 0, 8, 5, 0, 0, 6, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 4, 9, 0, 0, 5, 0],
        [2, 1, 3, 7, 0, 0, 0, 0, 0]
    ])

    #print("\nSolve in csv file:")
    #solve("puzzle.csv")

Answer:
 [[9 3 8 1 6 4 2 5 7]
 [5 7 4 2 9 3 1 8 6]
 [1 6 2 7 8 5 3 9 4]
 [8 4 5 3 7 6 9 2 1]
 [7 1 9 4 2 8 6 3 5]
 [6 2 3 5 1 9 4 7 8]
 [2 8 7 6 3 1 5 4 9]
 [4 9 6 8 5 2 7 1 3]
 [3 5 1 9 4 7 8 6 2]]

Masked anwser:
 [[9 0 8 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [1 0 0 7 8 0 0 0 0]
 [0 4 5 3 0 0 0 0 1]
 [7 1 0 4 2 0 0 3 5]
 [6 2 0 0 0 0 0 7 8]
 [0 0 7 6 0 0 0 0 9]
 [0 0 6 0 0 0 0 0 0]
 [0 5 0 0 4 0 0 6 0]]

Trail:
 [[9 7 8 2 1 5 3 4 6]
 [5 6 2 9 3 4 1 8 7]
 [1 3 4 7 8 6 5 9 2]
 [8 4 5 3 6 7 9 2 1]
 [7 1 9 4 2 8 6 3 5]
 [6 2 3 5 9 1 4 7 8]
 [4 8 7 6 5 3 2 1 9]
 [3 9 6 1 7 2 8 5 4]
 [2 5 1 8 4 9 7 6 3]]

Checked: OK

Solve in code:

Trail:
 [[8 5 4 2 6 7 9 1 3]
 [9 2 7 1 3 8 4 6 5]
 [6 3 1 9 4 5 7 2 8]
 [5 6 9 3 7 2 1 8 4]
 [1 4 2 6 8 9 5 3 7]
 [3 7 8 5 1 4 6 9 2]
 [4 9 5 8 2 1 3 7 6]
 [7 8 6 4 9 3 2 5 1]
 [2 1 3 7 5 6 8 4 9]]


In [ ]:
def print_grid(arr):
    print("\n"+"-"*37)
    for i in range(9):
        for j in range(9):
            if (j==0):
                print("|",arr[i][j], end=" | ")
            else:
                print(arr[i][j], end=" | ")
        print("\n"+"-"*37)
 
def find_empty_location(arr, l):
    for row in range(9):
        for col in range(9):
            if(arr[row][col]== 0):
                l[0]= row
                l[1]= col
                return True
    return False
 
def used_in_row(arr, row, num):
    for i in range(9):
        if(arr[row][i] == num):
            return True
    return False

def used_in_col(arr, col, num):
    for i in range(9):
        if(arr[i][col] == num):
            return True
    return False
 
def used_in_box(arr, row, col, num):
    for i in range(3):
        for j in range(3):
            if(arr[i + row][j + col] == num):
                return True
    return False
 

def check_location_is_safe(arr, row, col, num):
     
    return not used_in_row(arr, row, num) and not used_in_col(arr, col, num) and not used_in_box(arr, row - row % 3, col - col % 3, num)
 
def solve_sudoku(arr):
     
    l =[0, 0]
     
    if(not find_empty_location(arr, l)):
        return True
     
    row = l[0]
    col = l[1]
     
    for num in range(1, 10):
         
        if(check_location_is_safe(arr, row, col, num)):
             
            arr[row][col]= num
 
            if(solve_sudoku(arr)):
                return True
 
            arr[row][col] = 0
             
    return False
 
if __name__=="__main__":
     
    grid =[[0 for x in range(9)]for y in range(9)]
     
    grid =[[3, 0, 6, 5, 0, 8, 4, 0, 0],
          [5, 2, 0, 0, 0, 0, 0, 0, 0],
          [0, 8, 7, 0, 0, 0, 0, 3, 1],
          [0, 0, 3, 0, 1, 0, 0, 8, 0],
          [9, 0, 0, 8, 6, 3, 0, 0, 5],
          [0, 5, 0, 0, 9, 0, 6, 0, 0],
          [1, 3, 0, 0, 0, 0, 2, 5, 0],
          [0, 0, 0, 0, 0, 0, 0, 7, 4],
          [0, 0, 5, 2, 0, 6, 3, 0, 0]]
          
    """grid =[[5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]]"""

    
    print("\n"+"Sudoku Solver".center(37, "*")+"\n")
    if(solve_sudoku(grid)):
        print_grid(grid)
    else:
        print("No solution exists...".center(37))


************Sudoku Solver************


-------------------------------------
| 3 | 1 | 6 | 5 | 7 | 8 | 4 | 9 | 2 | 
-------------------------------------
| 5 | 2 | 9 | 1 | 3 | 4 | 7 | 6 | 8 | 
-------------------------------------
| 4 | 8 | 7 | 6 | 2 | 9 | 5 | 3 | 1 | 
-------------------------------------
| 2 | 6 | 3 | 4 | 1 | 5 | 9 | 8 | 7 | 
-------------------------------------
| 9 | 7 | 4 | 8 | 6 | 3 | 1 | 2 | 5 | 
-------------------------------------
| 8 | 5 | 1 | 7 | 9 | 2 | 6 | 4 | 3 | 
-------------------------------------
| 1 | 3 | 8 | 9 | 4 | 7 | 2 | 5 | 6 | 
-------------------------------------
| 6 | 9 | 2 | 3 | 5 | 1 | 8 | 7 | 4 | 
-------------------------------------
| 7 | 4 | 5 | 2 | 8 | 6 | 3 | 1 | 9 | 
-------------------------------------
